In [1]:
# Importing Required Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)# To see all the columns of a dataframe
#pd.set_option('display.max_rows', None)

In [2]:
# Loadiing reduced feature training set
X_train = pd.read_csv('X_train_final.csv')
y_train = pd.read_csv('y_train.final.csv')

In [3]:
# Loadiing reduced feature test set
X_test = pd.read_csv('X_test_final.csv')
y_test = pd.read_csv('y_test.final.csv')

## Model_16: Weighted Aggregating Classifier 

In [4]:
# Importing Joblib module
import joblib

In [5]:
import tensorflow as tf
from tensorflow import keras

In [6]:
# Importing best Voting Classifier i.e. voting Classifier_3
voting_clf = joblib.load('Voting_class_3.joblib')

In [7]:
# Importing best Neural Net Classifier i.e. Dense Neural Net with RELU activation and Nadam optimizer
neural_clf = keras.models.load_model('Best_model_Elu_eq_Learn_relu.h5')

### Performing 5 fold class validations to get best weights for Voting and Neural Calssifiers 

In [8]:
from sklearn.model_selection import StratifiedKFold

In [9]:
# Initializing the 5 Fold  Stratified fold object
cv_strat = StratifiedKFold(5,random_state=42)

In [10]:
# Importing the Sklearn's roc_auc_score module
from sklearn.metrics import roc_auc_score

In [11]:
# Importing cross val score from sklearn
from sklearn.model_selection import cross_val_score

In [12]:
# Importing train test split from Sklearn to produce validation set
from sklearn.model_selection import train_test_split

In [13]:
# Instantiating an empty DataFrame to store predictions of the Voting & Neural classifier on the test fold
df_pred = pd.DataFrame(columns=['Voting','Neural','True'])

In [14]:
# Defining the exponential decay learning rate with tuned step parameter = 2 for neural classifier with RELU activation
def exponential_decay_fn(epoch):
            return 0.01 * 0.1**(epoch /2 )

In [15]:
def Neural_Pred(clf, X_tr, y_tr, X_tst):
    ''' 
    This function returns the predictions of Neural Classifier(clf) for each of the test fold. 
    Parameters:
    X_tr : Training set Features
    y_tr : Training set Labels
    X_tst : Test set Features
   
    '''
    # Splitting the Training set further into training & validation set.
    X_tr_r, X_val, y_tr_r, y_val = train_test_split(X_tr, y_tr, test_size=0.15, random_state=42, stratify=y_tr)
    
    # Compiling & Training the Neural Net
    clf.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Nadam(beta_1=0.9, beta_2=0.999),
                metrics=[tf.keras.metrics.AUC(curve="ROC")])
    
    # defining Checkpoints
    checkpoint_cb = keras.callbacks.ModelCheckpoint('best_model.h5',save_best_only=True) # 1st Callback
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=4,restore_best_weights=True)# 2nd Callback
    lr_scheduler_cb = keras.callbacks.LearningRateScheduler(exponential_decay_fn)# 3rd Callback
    
    # Fitting The model
    clf.fit(X_tr_r, y_tr_r, epochs=50, validation_data=(X_val,y_val),batch_size=32,
    class_weight={0: 1.0, 1: 12.0},callbacks=[checkpoint_cb,early_stopping_cb,lr_scheduler_cb])
    
    # Loading the best Neural model after training & before making predictions
    clf_best = keras.models.load_model('best_model.h5')    
    
    # returning the predictions for the test fold.
    return (clf_best.predict_proba(X_tst).flatten())

In [16]:
# Computing Voting Classifier's & Neural Net predictions on the test fold
for train_index, test_index in cv_strat.split(X_train, y_train):
    # Creating  Folds
    X_tr, X_tst = X_train.loc[train_index], X_train.loc[test_index]
    y_tr, y_tst = y_train.loc[train_index], y_train.loc[test_index]
    
    # Fitting the best Voting Classifier on the Training Folds
    voting_clf.fit(X_tr.values, y_tr)
    # Making Predictions on the testing Fold.
    y_pred_voting = voting_clf.predict_proba(X_tst.values)[:,1]
    
    
    # Loading the fresh best Neural net classifier
    neural = keras.models.load_model('Best_model_Elu_eq_Learn_relu.h5')
    # Fitting the best Neural Net on Training Folds & obtaining  predictions
    y_pred_neural = Neural_Pred(neural, X_tr, y_tr, X_tst)
    
    # Storing the predictions in a dataframe
    df_temp = pd.DataFrame(columns=['Voting','Neural','True'])
    df_temp['Voting'] = y_pred_voting
    df_temp['Neural'] = y_pred_neural
    df_temp['True'] = y_tst.values.flatten() # As y_tst is a dataframe we first convert it into a 2d numpy vector
    # array, which is is then converted to 1d array using .flatten()
    
    # Appending the df_temp to df_train_stack
    df_pred = df_pred.append(df_temp,ignore_index=True)   
    

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 156830 samples, validate on 27676 samples
Epoch 1/50
156830/156830 [==============================] - 24s 150us/sample - loss: 1.1223 - auc: 0.7484 - val_loss: 1.4785 - val_auc: 0.7539
Epoch 2/50
156830/156830 [==============================] - 22s 139us/sample - loss: 1.1015 - auc: 0.7598 - val_loss: 4.0262 - val_auc: 0.7607
Epoch 3/50
156830/156830 [==============================] - 22s 140us/sample - loss: 1.0917 - auc: 0.7648 - val_loss: 2.3824 - val_auc: 0.7563
Epoch 4/50
156830/156830 [==============================] - 22s 142us/sample - loss: 1.0849 - auc: 0.7688 - val_loss: 4.3395 - val_auc: 0.7558
Epoch 5/50
156830/156830 [==============================] - 22s 141us/sample - loss: 1.0815 - auc: 0.7705 - val_loss: 3.5095 - val_auc: 0.7565
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 156830 samples, validate on 27676 samples
Epoch 1/50
156830/156830 [==============================] - 25s 158us/sample - loss

In [17]:
# Visualizing the head of the df_pred
df_pred.head()

,Voting,Neural,True
0,0.453474,0.583205,0
1,0.114683,0.179734,0
2,0.109974,0.177286,0
3,0.472716,0.512960,1
4,0.439749,0.525648,1


In [19]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230633 entries, 0 to 230632
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Voting  230633 non-null  float64
 1   Neural  230633 non-null  float32
 2   True    230633 non-null  object 
dtypes: float32(1), float64(1), object(1)
memory usage: 4.4+ MB


In [21]:
# Converting the True column to int
df_pred['True'] = df_pred["True"].astype(np.int)

In [22]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230633 entries, 0 to 230632
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Voting  230633 non-null  float64
 1   Neural  230633 non-null  float32
 2   True    230633 non-null  int32  
dtypes: float32(1), float64(1), int32(1)
memory usage: 3.5 MB


In [23]:
# importing hyper-parameter tuner optuna
import optuna

In [24]:
# Importing the Sklearn's roc_auc_score module
from sklearn.metrics import roc_auc_score

In [25]:
# Declaring objective function for Weighted Aggregation Classifier
def objective_wrappper_auc(y_pred_cls_1, y_pred_cls_2, y_true):
    '''
    Optimizes weights for predictions from 2 classifiers to achieve maximum roc_auc score
      
    '''
    
    def objective(trial):
        w1 = trial.suggest_uniform('w1',-1,1)
        w2 = trial.suggest_uniform('w2',-1,1)
        
        y_final_pred = y_pred_cls_1 * w1 + y_pred_cls_2 * w2
        
        roc_auc_1 = roc_auc_score(y_true,y_final_pred)
        
                   
        return roc_auc_1
    
    return objective

In [29]:
# Defining the evaluation function for study's best parameters
def roc_auc_weights(y_pred_cls_1, y_pred_cls_2, y_true, obj_func, n_trials=200):
    ''' Computes the best hyper parameters of the classsifier on the Training set and returns 
    Optuna's study's best score & clasifier parameters'''
    study = optuna.create_study(direction='maximize')
    study.optimize(obj_func(y_pred_cls_1, y_pred_cls_2, y_true),n_trials)
    best_score = study.best_value
    best_params = study.best_params
    return (best_score,best_params)


In [31]:
best_study_score,best_study_params = roc_auc_weights(df_pred['Voting'].values, df_pred['Neural'].values, df_pred['True'].values, 
                                                     objective_wrappper_auc,n_trials=2000)

[I 2021-01-09 02:44:06,427] A new study created in memory with name: no-name-220e5526-1422-4220-b024-fc875062bcb3
[I 2021-01-09 02:44:06,485] Trial 0 finished with value: 0.7620881569365854 and parameters: {'w1': 0.22822449452402727, 'w2': 0.5357813070274884}. Best is trial 0 with value: 0.7620881569365854.
[I 2021-01-09 02:44:06,537] Trial 1 finished with value: 0.6123382863184641 and parameters: {'w1': 0.47704777513389085, 'w2': -0.41782190575282896}. Best is trial 0 with value: 0.7620881569365854.
[I 2021-01-09 02:44:06,589] Trial 2 finished with value: 0.4633859620054448 and parameters: {'w1': 0.8857141337655585, 'w2': -0.947698904818205}. Best is trial 0 with value: 0.7620881569365854.
[I 2021-01-09 02:44:06,639] Trial 3 finished with value: 0.7496537260482155 and parameters: {'w1': -0.04835498516090819, 'w2': 0.35807712330755703}. Best is trial 0 with value: 0.7620881569365854.
[I 2021-01-09 02:44:06,691] Trial 4 finished with value: 0.23610975631867714 and parameters: {'w1': -0.

[I 2021-01-09 02:44:08,660] Trial 39 finished with value: 0.7684869697356086 and parameters: {'w1': 0.8465925379266503, 'w2': 0.26868551372224525}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:08,717] Trial 40 finished with value: 0.7686708461343559 and parameters: {'w1': 0.9908457367024549, 'w2': 0.043478086221006865}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:08,774] Trial 41 finished with value: 0.7687699435689779 and parameters: {'w1': 0.7420479034998689, 'w2': 0.13391923617305265}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:08,831] Trial 42 finished with value: 0.7687692694695124 and parameters: {'w1': 0.7379560806034049, 'w2': 0.13373931423184213}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:08,888] Trial 43 finished with value: 0.7683968828398404 and parameters: {'w1': 0.5608719425752431, 'w2': -0.010482100635903158}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-0

[I 2021-01-09 02:44:10,955] Trial 79 finished with value: 0.766008008343196 and parameters: {'w1': 0.3137397521234575, 'w2': 0.3017455659148917}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:11,009] Trial 80 finished with value: 0.7687843922488414 and parameters: {'w1': 0.6159484079869095, 'w2': 0.1003016269325396}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:11,064] Trial 81 finished with value: 0.7687911081642609 and parameters: {'w1': 0.587856180811245, 'w2': 0.08826770557515581}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:11,119] Trial 82 finished with value: 0.7686837117824419 and parameters: {'w1': 0.6987537737939231, 'w2': 0.1658866196220564}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:11,174] Trial 83 finished with value: 0.768097223208088 and parameters: {'w1': 0.5445774356997644, 'w2': 0.23678853002858685}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:1

[I 2021-01-09 02:44:13,156] Trial 118 finished with value: 0.768338320037117 and parameters: {'w1': 0.5243459630546169, 'w2': -0.014687923673378483}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:13,211] Trial 119 finished with value: 0.7674468674459267 and parameters: {'w1': 0.5932926731892919, 'w2': -0.07160462123085357}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:13,266] Trial 120 finished with value: 0.7687725936082523 and parameters: {'w1': 0.6643813136269865, 'w2': 0.0630824677007488}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:13,321] Trial 121 finished with value: 0.7687830807821248 and parameters: {'w1': 0.7669538135957695, 'w2': 0.12631376643597786}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:13,376] Trial 122 finished with value: 0.7686580643370491 and parameters: {'w1': 0.7189715382736738, 'w2': 0.028425599485914096}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-

[I 2021-01-09 02:44:15,342] Trial 157 finished with value: 0.7686342256365581 and parameters: {'w1': 0.6976288136320786, 'w2': 0.18166905373161368}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:15,397] Trial 158 finished with value: 0.7681079262153858 and parameters: {'w1': 0.7786998685881652, 'w2': -0.04604467889564315}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:15,452] Trial 159 finished with value: 0.7687913885957183 and parameters: {'w1': 0.734106503444008, 'w2': 0.1093865177034191}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:15,510] Trial 160 finished with value: 0.7686969870580498 and parameters: {'w1': 0.9608106537349438, 'w2': 0.22163578388541044}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-09 02:44:15,566] Trial 161 finished with value: 0.768792947814888 and parameters: {'w1': 0.8059286970139098, 'w2': 0.11645125266647632}. Best is trial 31 with value: 0.7687940520612504.
[I 2021-01-0

...

[I 2021-01-09 02:45:56,631] Trial 1834 finished with value: 0.3134310987784861 and parameters: {'w1': -0.20845110020774224, 'w2': 0.1557978881303941}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:45:56,692] Trial 1835 finished with value: 0.7687795101575601 and parameters: {'w1': 0.6676333709105513, 'w2': 0.11321049511585665}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:45:56,753] Trial 1836 finished with value: 0.7686710698208754 and parameters: {'w1': 0.7374375387551302, 'w2': 0.03241186448753296}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:45:56,815] Trial 1837 finished with value: 0.7687836710308112 and parameters: {'w1': 0.8112974530113342, 'w2': 0.08643440034506852}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:45:56,878] Trial 1838 finished with value: 0.768766407396697 and parameters: {'w1': 0.727243036201539, 'w2': 0.13392815675748848}. Best is trial 1387 with value: 0.768794222802716.
[

[I 2021-01-09 02:45:59,086] Trial 1873 finished with value: 0.7687883410885517 and parameters: {'w1': 0.8077971185162854, 'w2': 0.12608960655159124}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:45:59,148] Trial 1874 finished with value: 0.76872833052993 and parameters: {'w1': 0.6548267691092625, 'w2': 0.043817161073282776}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:45:59,211] Trial 1875 finished with value: 0.7686595521183949 and parameters: {'w1': 0.3999564469787861, 'w2': 0.09959163052208268}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:45:59,273] Trial 1876 finished with value: 0.7686812638970095 and parameters: {'w1': 0.7583716599588015, 'w2': 0.18094002485940927}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:45:59,334] Trial 1877 finished with value: 0.7686812849230358 and parameters: {'w1': 0.5690844273146979, 'w2': 0.1357633293377827}. Best is trial 1387 with value: 0.768794222802716.
[I

[I 2021-01-09 02:46:01,535] Trial 1912 finished with value: 0.7687063150139011 and parameters: {'w1': 0.6875202554870351, 'w2': 0.15515487393444283}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:46:01,598] Trial 1913 finished with value: 0.7687916626940354 and parameters: {'w1': 0.9644916515724345, 'w2': 0.11698020012564997}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:46:01,661] Trial 1914 finished with value: 0.7687085642920501 and parameters: {'w1': 0.8052816313461689, 'w2': 0.1805774840208384}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:46:01,723] Trial 1915 finished with value: 0.768692027195804 and parameters: {'w1': 0.8986356121750533, 'w2': 0.04635009217670094}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:46:01,786] Trial 1916 finished with value: 0.7687937951890753 and parameters: {'w1': 0.6272801729325924, 'w2': 0.08202025217280422}. Best is trial 1387 with value: 0.768794222802716.
[I

[I 2021-01-09 02:46:03,988] Trial 1951 finished with value: 0.7687924667495423 and parameters: {'w1': 0.9651541426971141, 'w2': 0.11976531863522744}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:46:04,051] Trial 1952 finished with value: 0.765425806544925 and parameters: {'w1': 0.8838982202447101, 'w2': 0.9885366650648874}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:46:04,113] Trial 1953 finished with value: 0.768784376036002 and parameters: {'w1': 0.9354448359758526, 'w2': 0.15224032747803118}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:46:04,177] Trial 1954 finished with value: 0.7686770543852398 and parameters: {'w1': 0.8660932012098065, 'w2': 0.20842414498921302}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:46:04,240] Trial 1955 finished with value: 0.7687803869175187 and parameters: {'w1': 0.9090261359860009, 'w2': 0.0933509255275164}. Best is trial 1387 with value: 0.768794222802716.
[I 2

[I 2021-01-09 02:46:06,453] Trial 1990 finished with value: 0.7687837883205717 and parameters: {'w1': 0.8573812627033924, 'w2': 0.09142976608792144}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:46:06,516] Trial 1991 finished with value: 0.7687917820104009 and parameters: {'w1': 0.9340449296988401, 'w2': 0.1383305598960673}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:46:06,577] Trial 1992 finished with value: 0.7687085554256534 and parameters: {'w1': 0.9984885028581579, 'w2': 0.058022226540416104}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:46:06,640] Trial 1993 finished with value: 0.7687937341376018 and parameters: {'w1': 0.8467266081838606, 'w2': 0.11845776978573022}. Best is trial 1387 with value: 0.768794222802716.
[I 2021-01-09 02:46:06,702] Trial 1994 finished with value: 0.7687517710025515 and parameters: {'w1': 0.8918832313321212, 'w2': 0.17563848211687}. Best is trial 1387 with value: 0.768794222802716.
[I 

In [32]:
print('The best roc_auc_score for the study is: ',best_study_score)

The best roc_auc_score for the study is:  0.768794222802716


In [33]:
print(('The best study parameters for the classifier are: ',best_study_params))

('The best study parameters for the classifier are: ', {'w1': 0.9675154320896099, 'w2': 0.1304126470684431})


### Training the Tuned Weighted Aggregating Classifier on the whole Training set & making the predictions on the Test set.

In [34]:
# Training the voting classifier on whole Traing set
voting_clf.fit(X_train.values, y_train)

VotingClassifier(estimators=[('logistic_Reg',
                              LogisticRegression(C=0.0031160262723627184,
                                                 class_weight={0: 1.0, 1: 14},
                                                 l1_ratio=0.4046164083668398,
                                                 n_jobs=5, penalty='elasticnet',
                                                 random_state=42,
                                                 solver='saga')),
                             ('Random_Forest',
                              RandomForestClassifier(class_weight={0: 1.0,
                                                                   1: 13},
                                                     max_depth=18,
                                                     min_samples_leaf=0.0010028580411287713,
                                                     n_estimators=570, n_jobs=5,
                                                     ra...
             

In [35]:
# Making Predictions on the Test Set using voting classifier
y_preds_voting = voting_clf.predict_proba(X_test.values)[:,1]

In [36]:
# Importing best Neural Net Classifier i.e. Dense Neural Net with RELU activation and Nadam optimizer
neural_clf = keras.models.load_model('Best_model_Elu_eq_Learn_relu.h5')

In [39]:
# Making Predictions on the Test Set using Neural classifier
y_preds_neural = neural_clf.predict_proba(X_test)

In [43]:
# combining the predictions from the Voting  & Neural classifier with the study's best votes.
y_combined_final = (y_preds_voting.reshape(-1,1)) * best_study_params['w1'] + y_preds_neural * best_study_params['w2']

In [43]:
print('The training set roc_auc score of the weighted Aggregating classifier is: ', best_study_score)

The training set roc_auc score of the weighted Aggregating classifier is:  0.768794222802716


In [44]:
# Computing the the final roc_auc score of the weighted Aggregating classifier on the test set.
print('The final test set roc_auc score of the combined voting & neural classifier using the best weights is: ',
     roc_auc_score(y_test,y_combined_final))

The final test set roc_auc score of the combined voting & neural classifier using the best weights is:  0.7726740445628326


In [45]:
# Saving the 5 fold Training set Predictions dataframe df_pred as a csv file.
df_pred.to_csv('Voting_Neural_Folds_Preds.csv',index=False)

### Calculating R_R ratio of the tuned Weighted Aggregating classifier using study's best weights.

In [46]:
# Calculating the length of each test fold
len_fold = len(df_pred)//5

In [47]:
len_fold

46126

In [48]:
best_study_params['w1']

0.9675154320896099

In [49]:
best_study_params['w2']

0.1304126470684431

In [55]:
# Calculating roc_auc score for each of the 5 folds and storing it to a list roc_auc_agg
roc_auc_agg = []
for i in range(5):
    if i == 4:
        y_pred_combined = (df_pred.iloc[len_fold*i:,0]* best_study_params['w1'] + 
                           df_pred.iloc[len_fold*i:,1]* best_study_params['w2'])
        roc_auc_agg.append(roc_auc_score(df_pred.iloc[len_fold*i:,2],y_pred_combined))
        
        
        
    else: 
        y_pred_combined = (df_pred.iloc[len_fold*i:len_fold*(i+1),0]* best_study_params['w1'] + 
                           df_pred.iloc[len_fold*i:len_fold*(i+1),1]* best_study_params['w2'])
        
        roc_auc_agg.append(roc_auc_score(df_pred.iloc[len_fold*i:len_fold*(i+1),2],y_pred_combined)) 

In [56]:
print('The reward associated with the Aggregating classifier using roc_auc metric is: ',np.mean(roc_auc_agg))

The reward associated with the Aggregating classifier using roc_auc metric is:  0.7689233205883108


In [57]:
print('The risk associated with the Aggregating Classifier using roc_auc metric is: ',np.std(roc_auc_agg))

The risk associated with the Aggregating Classifier using roc_auc metric is:  0.004576671065945797


In [58]:
R_R_Ratio_Aggregator = np.mean(roc_auc_agg)/np.std(roc_auc_agg)

In [59]:
print('The reward risk ratio for the Aggregating Classifier using roc_auc metric is: ',R_R_Ratio_Aggregator)

The reward risk ratio for the Aggregating Classifier using roc_auc metric is:  168.00930403535745


In [60]:
print('5 fold CV roc_auc scores for the tuned Aggregating Classifier are: ',roc_auc_agg)

5 fold CV roc_auc scores for the tuned Aggregating Classifier are:  [0.763284109827825, 0.764284166520123, 0.7753680619254809, 0.7719635459532233, 0.7697167187149022]


In [63]:
# Saving the test set predictions of the Weighted Aggregating classifier as a dataframe
df_weighted_test_preds = pd.DataFrame(y_combined_final.flatten(),columns=['Weighted_Test_Preds'])

In [64]:
# Saving the test set predictions of the Weighted Aggregating classifier as the csv file
df_weighted_test_preds.to_csv('Weighted_Classifier_Test_Preds.csv',index=False)

#### R_R Ratio for the Weighted Aggregating classifier  using roc_auc metric is:  168.00930403535745

## Observations:
### 1)  The Weighted Aggregating Classifier has fitted the Training set very well, with no signs of overfitting. Also its test set roc_auc score is the highest of any classifier tested till now on this dataset.
### 2) Its R_R ratio is also on the higher side of the spectrum. Thus this classifier is one of the contenders for the best classifier on this dataset.



## Conclusion: Considering all the aspects such as overfitting, test set roc_auc score, R_R ratio etc. , Weighted Aggregating Classifier is one of the best choices. Hence we would use it for making forecasts on future observations and in calculating the optimal threshold probability for this dataset.